# Attribution using daily scaling

This notebook is similar to the previous attribution notebook in that it calculates the probability ratios for an event.
However, it differs in how the counterfactual climates are represented.
In this notebook the counterfactual climate is instead represented by shifting the daily values based on the relationship between GMST and either the monthly median or monthly quantiles.
The method is adapted from [Gilford et. al. (2022)](https://ascmo.copernicus.org/articles/8/135/2022/).

Note that this method is experimental and requires further development.

# Attribution: Observations

In [ ]:
import glob
import os
from functools import partial

import attribution.funcs
import attribution.utils
import attribution.validation
import cartopy.crs as ccrs
import iris
import iris.plot as iplt
import numpy as np
import scipy.stats as scstats
from attribution.config import init_config
from climix.metadata import load_metadata
from dask.distributed import Client
from iris_utils.utils import get_weights
from matplotlib import pyplot as plt

In [ ]:
client = Client()

We load in the the configuration from the project `.yml`.

In [ ]:
CFG = init_config()

In [ ]:
# CFG

Fetch the prepared datasets

In [ ]:
files = glob.glob(CFG["paths"]["project_folder"] + f"/{CFG['variable']}*.nc*")

In [ ]:
files

In [ ]:
variable = CFG["variable"][0]
print(variable)

### Event definition
We want to check how many days above 25 degrees there was during 2018.
So before we can define the threshold we have to calculate the index.

In [ ]:
index_name = CFG["index_name"][0]
print(index_name)

In [ ]:
index_catalog = load_metadata()
index = index_catalog.prepare_indices([index_name])[0]

## GridClim
Load GridClim

In [ ]:
file = glob.glob(CFG["paths"]["project_folder"] + f"/{variable}*GridClim*.nc")

In [ ]:
# GridClim is the first file in the list.
iris.FUTURE.datum_support = True
cube = iris.load_cube(file)

Remove leap days. Their inclusion makes the quantile shifting problematic.

In [ ]:
# Add a leap day cooridnate.
iris.coord_categorisation.add_categorised_coord(
    cube,
    "leap",
    "time",
    lambda coord, value: coord.units.num2date(value).month == 2
    and coord.units.num2date(value).day == 29,
)
#  Create the constraint.
leap_constraint = iris.Constraint(leap=False)
# Here remove leap days
cube = cube.extract(leap_constraint)

And some coord. categorisation.

In [ ]:
# Add more time categorisation.
iris.coord_categorisation.add_month_number(cube, "time")
iris.coord_categorisation.add_year(cube, "time")

In [ ]:
cube_av = attribution.utils.compute_spatial_average(cube)

### Compute index

In [ ]:
index_cube = attribution.utils.compute_index(cube, index, client)

In [ ]:
index_cube_av = attribution.utils.compute_index(cube_av, index, client)

In [ ]:
index_cube_av.shape

Now we can get the threshold from the SU of 2018.

In [ ]:
# Summer 2018
threshold = index_cube_av[-1].data

In [ ]:
threshold

In [ ]:
iplt.plot(index_cube_av);

### Fitting an extreme value distribution to SU

We try and fit a number of extreme value distributions to the data.

In [ ]:
# Some distributions describing extremes.
dists = {
    "genextreme": scstats.genextreme,
    "gamma": scstats.gamma,
    "genpareto": scstats.genpareto,
    "gengamma": scstats.gengamma,
    "gumbel_l": scstats.gumbel_l,
    "gumbel_r": scstats.gumbel_r,
}

Before we do the bootstrap, we want to check the goodness of fit for the distribution and the data.
For this we use a Kolmogorov-Smirnof test (KS-test).
For a goodness of fit this is a bit unintuitive.
The 0-hypothesis is that the distributions are the same, hence we are looking for a high p-value here. e.g. that we can't say that the dists are different.

In [ ]:
cube_data = index_cube_av.data

In [ ]:
_ = attribution.validation.inspect_distributions(cube_data, dists, figsize=(7, 5))

Which distribution that fits the data best is a difficult question,
and one should do both a visual and a statistical inspection.
In this case, based on the KS-test and the plot above, the gamma distribution seems to represent the data the best.

We need the quantile of the threshold to compute the "relative" thresholds for other datasets.
Since different datasets have different biases, the absolute threshold for this dataset does not translate to other datasets.

In [ ]:
# Fit the distribution.
fit = dists["gumbel_r"].fit(cube_data)
# Get the quantile of the survival function.
threshold_quantile = dists["gumbel_r"].sf(threshold, *fit)

In [ ]:
threshold_quantile

Detrending data

In [ ]:
# Add more time categorisation.
# iris.coord_categorisation.add_month_number(cube_av, "time")
# iris.coord_categorisation.add_year(cube_av, "time")

In [ ]:
gmst  = attribution.utils.get_monthly_gmst(cube_av)

In [ ]:
monthly_gmst = gmst.reshape(-1, 12)

In [ ]:
gmst_diff = monthly_gmst[-1] - monthly_gmst

In [ ]:
from importlib import reload

In [ ]:
cube_detrend = cube_av.copy()

In [ ]:
betas, p_values = attribution.utils.compute_monthly_regression_coefs(cube_av, gmst, tqdm=True)

In [ ]:
cube_detrend = attribution.funcs.shift_cube_data(cube_av, betas, p_values, gmst_diff, tqdm=True)

In [ ]:
reload(attribution.funcs)

In [ ]:
reload(attribution.utils)

In [ ]:
cube_cf = attribution.funcs.shift_cube_data(cube_av, betas, p_values, -1.2, tqdm=True)

In [ ]:
iplt.plot(cube_av);
iplt.plot(cube_detrend);
iplt.plot(cube_cf);

In [ ]:
index_cube = attribution.utils.compute_index(cube_av, index, client)
index_cube_detrend = attribution.utils.compute_index(cube_detrend, index, client)
index_cube_cf = attribution.utils.compute_index(cube_cf, index, client)

In [ ]:
iplt.plot(index_cube, label="Orig.");
iplt.plot(index_cube_detrend, label="De-trend");
iplt.plot(index_cube_cf, label="CF");
plt.legend();

In [ ]:
iris.coord_categorisation.add_year(cube_detrend, "time")

In [ ]:
test_betas, _ = attribution.utils.compute_monthly_regression_coefs(cube_detrend, gmst, tqdm=True)

### Median shifted probabilities
For this we need the averaged cube.

In [ ]:
# Select the first 30 years of the data.
# cube_av = cube_av[-30*365:].copy()

In [ ]:
from importlib import reload

In [ ]:
reload(attribution.bootstrap)

In [ ]:
reload(attribution.funcs)

In [ ]:
reload(attribution.utils)

In [ ]:
iris.coord_categorisation.add_year(cube_av, "time")

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    delta_temp=-1.2,
    log_sf=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

Save the quantiles

In [ ]:
np.save(
    os.path.join(
        CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_gridclim_ds_med"
    ),
    prob_ratio_ci,
)

### Quantile shifted prob. ratio

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    delta_temp=-1.0,
    log_sf=True,
    quantile_shift=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

Save the quantiles

In [ ]:
np.save(
    os.path.join(
        CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_gridclim_ds_q"
    ),
    prob_ratio_ci,
)

## EOBS
Not as many explanations here.

In [ ]:
file = glob.glob(CFG["paths"]["project_folder"] + f"/{variable}*EOBS*.nc")

In [ ]:
eobs_cube = iris.load_cube(file)

Remove leap days

In [ ]:
# Add a leap day cooridnate.
iris.coord_categorisation.add_categorised_coord(
    eobs_cube,
    "leap",
    "time",
    lambda coord, value: coord.units.num2date(value).month == 2
    and coord.units.num2date(value).day == 29,
)
#  Create the constraint.
leap_constraint = iris.Constraint(leap=False)
# Here remove leap days
eobs_cube = eobs_cube.extract(leap_constraint)

And some coord. categorisation.

In [ ]:
# Add more time categorisation.
iris.coord_categorisation.add_month_number(eobs_cube, "time")
iris.coord_categorisation.add_year(eobs_cube, "time")

In [ ]:
cube_av = attribution.utils.compute_spatial_average(eobs_cube)

Summer data

In [ ]:
mjja_cube = attribution.utils.select_season(eobs_cube, "mjja", "summer")

In [ ]:
mjja_cube_av = attribution.utils.compute_spatial_average(mjja_cube)

Compute the index

In [ ]:
index_cube = attribution.utils.compute_index(mjja_cube, index, client)

In [ ]:
index_cube_av = attribution.utils.compute_index(mjja_cube_av, index, client)

In [ ]:
_ = attribution.validation.inspect_distributions(index_cube_av.data, dists)

In [ ]:
index_cube_av.data[-1]

In [ ]:
# Fit the distribution.
fit = dists["gumbel_r"].fit(index_cube_av.data)
# Get the quantile of the survival function.
threshold_quantile_t = dists["gumbel_r"].sf(index_cube_av.data[-1], *fit)

In [ ]:
threshold_quantile

In [ ]:
threshold_quantile_t

### Median shifted probability ratios

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    delta_temp=-1.2,
    log_sf=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

In [ ]:
np.save(
    os.path.join(
        CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_eobs_ds_med"
    ),
    prob_ratio_ci,
)

### Quantile shifted probability ratios

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    delta_temp=-1.0,
    log_sf=True,
    quantile_shift=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

In [ ]:
np.save(
    os.path.join(CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_eobs_ds_q"),
    prob_ratio_ci,
)

## ERA5

In [ ]:
file = glob.glob(CFG["paths"]["project_folder"] + f"/{variable}*era5*.nc")

In [ ]:
era_cube = iris.load_cube(file)

Remove leap days

In [ ]:
# Add a leap day cooridnate.
iris.coord_categorisation.add_categorised_coord(
    era_cube,
    "leap",
    "time",
    lambda coord, value: coord.units.num2date(value).month == 2
    and coord.units.num2date(value).day == 29,
)
#  Create the constraint.
leap_constraint = iris.Constraint(leap=False)
# Here remove leap days
era_cube = era_cube.extract(leap_constraint)

And some coord. categorisation.

In [ ]:
# Add more time categorisation.
iris.coord_categorisation.add_month_number(era_cube, "time")
iris.coord_categorisation.add_year(era_cube, "time")

In [ ]:
cube_av = attribution.utils.compute_spatial_average(era_cube)

Summer data

In [ ]:
mjja_cube = attribution.utils.select_season(era_cube, "mjja", "summer")

In [ ]:
mjja_cube_av = attribution.utils.compute_spatial_average(mjja_cube)

Compute the index

In [ ]:
index_cube = attribution.utils.compute_index(mjja_cube, index, client)

In [ ]:
index_cube_av = attribution.utils.compute_index(mjja_cube_av, index, client)

In [ ]:
_ = attribution.validation.inspect_distributions(index_cube_av.data, dists)

In [ ]:
iplt.plot(index_cube_av);

In [ ]:
# 2018 is not the last year in the ERA5 dataset.
index_cube_av.data[-4]

In [ ]:
# Fit the distribution.
fit = dists["genextreme"].fit(index_cube_av.data)
# Get the quantile of the survival function.
threshold_quantile_t = dists["genextreme"].sf(index_cube_av.data[-4], *fit)

In [ ]:
threshold_quantile

In [ ]:
threshold_quantile_t

### Median shifted probability ratios

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    delta_temp=-1.2,
    log_sf=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

In [ ]:
np.save(
    os.path.join(
        CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_era5_ds_med"
    ),
    prob_ratio_ci,
)

### Quantile shifted probability ratios

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    delta_temp=-1.0,
    log_sf=True,
    quantile_shift=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

In [ ]:
np.save(
    os.path.join(CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_era5_ds_q"),
    prob_ratio_ci,
)

# Attribution: Models
## CORDEX ENS

In [ ]:
# Which file is CORDEX?
files[2]

In [ ]:
cube = iris.load_cube(files[2])

In [ ]:
# Add a leap day cooridnate.
iris.coord_categorisation.add_categorised_coord(
    cube,
    "leap",
    "time",
    lambda coord, value: coord.units.num2date(value).month == 2
    and coord.units.num2date(value).day == 29,
)
#  Create the constraint.
leap_constraint = iris.Constraint(leap=False)
# Here remove leap days
cube = cube.extract(leap_constraint)

And some coord. categorisation.

In [ ]:
# Add more time categorisation.
iris.coord_categorisation.add_month_number(cube, "time")
iris.coord_categorisation.add_year(cube, "time")

In [ ]:
cube_av = attribution.utils.compute_spatial_average(cube)

### Median shifted probability ratio

In [ ]:
# Some distributions describing extremes.
dists = {
    # "genextreme": scstats.genextreme,
    "gamma": scstats.gamma,
    "genpareto": scstats.genpareto,
    "gengamma": scstats.gengamma,
    "gumbel_l": scstats.gumbel_l,
    "gumbel_r": scstats.gumbel_r,
}

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    ensemble=True,
    delta_temp=-1.0,
    log_sf=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

In [ ]:
np.save(
    os.path.join(
        CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_cordex_ds_med"
    ),
    prob_ratio_ci,
)

### Quantile shifted probability ratio

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    ensemble=True,
    delta_temp=-1.0,
    log_sf=True,
    quantile_shift=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

In [ ]:
np.save(
    os.path.join(
        CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_cordex_ds_q"
    ),
    prob_ratio_ci,
)

## S-Lens ENS

In [ ]:
# Which file is CORDEX?
files[4]

In [ ]:
cube = iris.load_cube(files[4])

In [ ]:
# Add a leap day cooridnate.
iris.coord_categorisation.add_categorised_coord(
    cube,
    "leap",
    "time",
    lambda coord, value: coord.units.num2date(value).month == 2
    and coord.units.num2date(value).day == 29,
)
#  Create the constraint.
leap_constraint = iris.Constraint(leap=False)
# Here remove leap days
cube = cube.extract(leap_constraint)

And some coord. categorisation.

In [ ]:
# Add more time categorisation.
iris.coord_categorisation.add_month_number(cube, "time")
iris.coord_categorisation.add_year(cube, "time")

In [ ]:
weights = get_weights(cube)
cube_av = cube.collapsed(["latitude", "longitude"], iris.analysis.MEAN, weights=weights)

### Median shifted probability ratio

In [ ]:
# Some distributions describing extremes.
dists = {
    # "genextreme": scstats.genextreme,
    "gamma": scstats.gamma,
    "genpareto": scstats.genpareto,
    "gengamma": scstats.gengamma,
    "gumbel_l": scstats.gumbel_l,
    "gumbel_r": scstats.gumbel_r,
}

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    ensemble=True,
    delta_temp=-1.0,
    log_sf=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

In [ ]:
np.save(
    os.path.join(
        CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_s-lens_ds_med"
    ),
    prob_ratio_ci,
)

### Quantile shifted probability ratio

In [ ]:
prob_ratio_ci, theta_hat_b = attribution.bootstrap.prob_ratio_ds_ci(
    cube_av,
    index,
    dists,
    threshold_quantile,
    ensemble=True,
    delta_temp=-1.0,
    log_sf=True,
    quantile_shift=True,
    season="mjja",
)

In [ ]:
prob_ratio_ci

In [ ]:
np.save(
    os.path.join(
        CFG["paths"]["project_folder"], f"etc/{index_name}-ann_pbr_s-lens_ds_q"
    ),
    prob_ratio_ci,
)

In [ ]:
client.shutdown()

## Next step

[Synthesis](./6_synthesis.ipynb)